# Lab Week 8

#### In the last lab, we:
- merged our paired end reads that overlapped using `bbmerge`
- mapped our reads to the reference genome using `bwa`
- converted to bam files and sorted for downstream processing with `samtools`

We did all this on small fastq files to make the programs run faster so we could see each of the individual steps. Over the weekend, I ran the full sequencing fastq files through this process to map the millions of reads contained in each file to the reference genome, a process that took many hours and many computer cores for each file. Today we will continue to process these files to identify SNPs from the sequencing reads. Ultimately this will allow us to count the reads supporting the reference and alternate alleles to estimate allele frequencies.

1) To get started, Log into spydur from the terminal tab: `ssh yournetid@spydur` 



2) Next we are going to start a job on the erickson partition of spydur so that you can use more memory and computer cores. Run the code below, which is the same as what we used last week to activate your session.

In [ ]:
srun --pty -t 3:00:00 --mem=40G --partition erickson --ntasks-per-node=3 bash

3) After this has started, navigate to your personal folder in the `~/shared_perickso` directory and make a new folder for `Lab_8` this week (remember, no spaces in file or folder names, this causes lots of problems in Unix! And make sure the folder is in your personal folder).

4) Now change directories into your `Lab_8` folder and use the shortcut below to create a variable for your working directory. This command will print your working directory and assign that full path to the variable `$WD`. Going forward, you'll be able to use `$WD` as a shortcut. You must be in your Lab_8 folder for this and downstream steps to work correctly! The second `pwd` command should display the full path to your Lab_8 folder.

In [ ]:
WD=`pwd`
echo $WD

5) Next, navigate to `~/shared_perickso/bam` and find the names of the final bam file for your two sequencing samples. Modify the code below to copy your samples to your working directory for this week. It will take about 15-20 sec to copy each file; they're big!

In [ ]:
cp <filename> $WD
cp <filename> $WD

6) Navigate back to your Lab_8 folder (you can use `cd $WD` as a shortcut) and make sure the bam files are there using `ls`. It is really important that you are working in this folder for the rest of the lab! 

7) Next we need to make a text file that records full paths of both of our working bam files that are in our `Lab_8` folder. Below, the code uses asterisk wildcard to represent any letter or number coming after "ZP". In other words `ZP*.bam` will match any filename that starts with `ZP`. So this command lists all the files in your folder that start with ZP and end with .bam. 

In [ ]:
ls ZP*.bam

Now use this same trick to save these full file names to a file called `sample_files.txt`

In [ ]:
ls $PWD/ZP*.bam > sample_files.txt

**Question: Look at the `sample_files.txt` that you just made above using `less`. What did that code that you just ran do?**

8) Now we need to make a second file that contains just the names of our two samples on separate lines. We'll be using the UNIX tool `echo`, which basically means "print". We will tell it to print our two file names with a line break in between them. The special character `\n` says to start a new line. Look at the command below and edit it for your file names. Inside the quotes, replace with the two names of your samples but leave the `\n`, for example `"ZP_1\nZP_2"`, but subbing your sample names for the 1 and 2.

In [ ]:
echo -e "first_sample\nsecond_sample" > sample_names.txt

9) Look at your `sample_names.txt` file with `less` to make sure it did what you wanted it to (each sample names should be on a separate line with no extra characters. If this file is not correct, the following steps will not work!

10) We need to create an index file for each of our mapped bam files; this is like the index of a book, which tells you where to look for certain information in the book. The index file helps the computer find reads in different parts of the genome more quickly. Edit the code to generate an index file for each of your mapping bam files. 

In [ ]:
samtools index <bam_file_1>
samtools index <bam_file_2>

11) For our first analysis we are going to find out how much sequencing data we actually successfully mapped to the genome in the bam files. The program we'll use is a function of samtools called `samtools depth`. The general format is `samtools depth input.bam > output.txt`

Edit the general code below to update with the file names of your input file name (the one file you worked with last week; your partner will do the other), and name the output file "samplename_depth.txt" (but update with the actual name of your sample, ZP_#!). You and your partner can each calculate depth for the file you were working with last week. This calculation might take a couple of minutes, but you can start reading ahead.

In [ ]:
samtools depth input.bam > sample_name_depth.txt

12) The file you just made has one line for every position in the genome (so ~150,000,000 lines) and the third column  contains the number of sequencing reads mapped to each position.  We can calculate the average depth of coverage for each scaffold using something called a **"for loop"** in unix. A **for loop** takes a variable and repeats calculations over a range of values for that variable. In this case, the code is taking a variable called `i` and executing the code for `i=1`, `i=2`...`i=5`. The code below uses a program called `awk` to calculate an average of all the numbers found in column 3.  Edit your sample_name_depth.txt file name, then copy this whole block of code and run it. 

In [ ]:
for i in {1..5}; do
echo "calculating depth for scaffold " $i
   y=`grep Scaffold_${i} sample_name_depth.txt | awk '{sum+=$3} END { print sum/NR}'`
    echo "the average depth of scaffold" $i "is" $y
    done  

**Question: why did the code loop through the numbers 1 to 5? Hint: where does the variable `{i}` show up in the code and what is that line doing?**

**Question: Can you think of a reason one chromosome might have lower coverage than the others? Hint: think about what chromosome is not present equally in all individuals in most species, and how that would be reflected when pooling many individuals together.**

13) Next we are going to generate what is called a "pileup file". This file takes all the mapping information from the bam files and summarizes it for every position in the genome. We are just going to create it for a small region of the genome (Scaffold_1:10,000,000-11,000,000) to speed up our future analysis. Complete the code below, assigning  a name for your pileup file and start running it, then answer the questions below while it runs. 
The first line, `cd $WD` will make sure you are in your Lab_8 folder! The second line is telling the program where to find the reference genome, and the third line uses `samtools` to generate the pileup file. **Make sure you change the name of the output .pileup file to match your sample names.**

In [ ]:
cd $WD
REFERENCE=~/shared_perickso/zind_ref_genome.fasta

#make mpileup file of your data--edit the name of the pileup file below to reflect your sample names
samtools mpileup -r Scaffold_1:10000000-11000000 -B -f $REFERENCE --bam-list sample_files.txt > sample_1_sample_2.pileup


14) While the program is running, go to this website: http://www.htslib.org/doc/samtools-mpileup.html and read about the pileup format to answer the following questions:

**Question:  Approximately how many lines will you expect to have in your pileup file given the region of the genome that we are looking at?**

**Question: How are bases that match the reference base indicated in the pileup file? How are SNPs indicated?**


15) After your code has run, look at your mpileup file using `less -S` and answer the following question:

**Question: Given what you learned above, where do you see the first SNP and how do you know?** 

**Question: Now count the number of lines in your pileup file with `wc -l <filename>`. Does it meet your expectation?**

16) Now we will actually call SNPs and indels in this small region of the genome. The code below uses a program called `Varscan` to actually identify variants using the pileup file. We set filters so that it requires variants to have a coverage of at least 10 reads and a minimum allele frequency of 0.05. The p-value filter refers to a statistical probability that the SNP is real. The SNPs and indels are called with two separate commands `mpileup2snp` and `mpileup2indel`. To run the code, you  just need to replace `sample1_sample2.pileup` with the name of the pileup file that you generated above on the first line of each block of code. Then copy and paste everything to your terminal to run and then proceed to the next step while it is running.

In [ ]:
#tell the computer how much memory to use
JAVAMEM=40g

#call SNPs
java -Xmx$JAVAMEM -jar /usr/local/sw/varscan/VarScan.v2.3.9.jar mpileup2snp sample_1_sample_2.pileup \
  --output-vcf 1 \
  --min-coverage 10 \
  --min-var-freq 0.05 \
  --p-value 0.1 \
  --vcf-sample-list sample_names.txt > raw_snps.vcf
  
#call indels

java -Xmx$JAVAMEM -jar /usr/local/sw/varscan/VarScan.v2.3.9.jar mpileup2indel sample_1_sample_2.pileup \
  --output-vcf 1 \
  --min-coverage 10 \
  --min-var-freq 0.05 \
  --p-value 0.1 \
  --vcf-sample-list sample_names.txt > raw_indels.vcf

**Question: While the code is running, go to https://www.ebi.ac.uk/training/online/courses/human-genetic-variation-introduction/variant-identification-and-analysis/understanding-vcf-format/ to learn about the output file, vcf files. Complete the activity and take a screenshot when you are done. Paste your screenshot here. It will just show up as a link to a jpg file but will be inserted when you export to PDF at the end**

**Question: Read the output that printed to the terminal when your code was done. How many SNPs and how many indels did it identify?**

17) The first several lines of the vcf file begin with # and have important information about the data that can be found in the file. Use `less -S raw_snps.vcf` to look at your raw SNP calls vcf file to answer the following questions. They will be very difficult to read if you don't add the -S to make things print in nice columns. If you are stuck on these questions, please talk to me because I want to be sure you understand this information!  

**Question: the vcf will store a lot of information about each variant in each sample. The key to decoding this information is provided in the header lines that start with ##FORMAT=. Look at the #FORMAT lines of your vcf. What are the names used in the format column to indicate the number of reference reads and alternate reads?**


**Question: The true header line of the vcf file that names the columns starts with ##CHROM. From left to right, where are the columns located that actually contain the data on your two samples?**


**Question: Look at the first SNP in your sample. Where in the genome is it located?**


**Question: Now keep reading across to the column that starts with ` GT:GQ:SDP...`. This column tells you the order of information that you will read about each sample, separated by a colon ( `:`) between each piece of information. You identified the abbreviations for reference and alternate read counts in a question above. Which positions in this series of information will have your reference and alternate read counts?**

**Question: Now keep reading across to your first sample. How many reads were found for the reference and alternate allele? These are the numbers in the positions you identified above.**


**Question: the vcf file format automatically assigns a "genotype" to each sample. Why does it not make sense to assign genotypes in pooled sequencing?**


18) Now that we've gotten oriented to the information in a vcf file, we need to do further filtering to only use high-confidence SNPs. As mentioned previously, indels cause quite a few problems for SNP calling, so we want to get rid of the SNPs that are located near indels. Varscan has a built in filtering program that can remove these. Run the code below to see what happens.

In [ ]:
java -Xmx$JAVAMEM -jar /usr/local/sw/varscan/VarScan.v2.3.9.jar filter raw_snps.vcf \
 --min-var-freq 0.05 \
 --indel-file  raw_indels.vcf \
 --output-file snps_no_indels.vcf

**Question: How many SNPs were removed because they were near indels and how many SNPs remain in your file? This information will be displayed when the code is done running.**

19) We ran all of these commands on just a tiny region of the genome to make things easier for us. I ran Varscan on all of our samples over the weekend. Using all of the samples together increases the ability to find variants relative to using just two samples because there is so much more information to compare. But, the resulting vcf file has 34 sequencing samples and is quite large, so it is currently stored in a compressed .gz format. To make these easier to work with, you are going to extract just the two samples you are working with from each of these variant files. We will use a program called `bcftools` to do this with a command called `view`. This program will use the list of sample names that you already made (`sample_names.txt`) to extract just those samples from the vcf file and save a smaller file. 

In [ ]:
bcftools view -S sample_names.txt ~/shared_perickso/ZP_all.vcf.gz > subset_snps.vcf

20) Now confirm that the previous command did what you wanted it to by checking the sample names present in the file using the command below. `query -l` is a shortcut that extracts all the sample names from a vcf. 

In [ ]:
bcftools query -l subset_snps.vcf

21) We also don't want to include SNPs in regions of the genome that contain repetitive sequences like retrotransposons, because mapping reads to these regions is imprecise and calling SNPs is not accurate. 

**Question: Why would it be hard to map reads and call SNPs in repetitive regions? (hint: think about the definition of repetitive and how read mapping works)**

22) I have a file that contains the positions of all of the repetitive sequences in the _Z. indianus_ genome. The file is in a `gff` format, which is basically just a table of chromsomes, start positions, and stop positions for regions of the genome. We can use a program called `bedtools` to remove all of these regions from our vcf file as well. This program essentially does a "subtraction" where it will remove any SNPs found within regions defined in the `~/shared_perickso/zind_ref_genome_repeats.gff` file. Run the code below to remove the SNPs in repetitive regions. The `\` after each line indicates that more code is coming. 


In [ ]:
bedtools intersect -v -header \
    -a subset_snps.vcf \
    -b ~/shared_perickso/zind_repeats.gff \
    > subset_snps_no_repeats.vcf

**Question: Use `wc -l` to count the number of lines in your starting file and the new file you produced in the command above. How many lines are in each and approximately how many SNPs were removed due to being in repetitive regions?**

23) Some SNPs are just going to be hard to work with. They might have more than two alleles. They may not have very many reads supporting them or they may have way too many reads supporting them, suggesting a potential genome duplication. We want to get rid of those SNPs too. The command below will filter any SNPs that have less than 15 or more than 200 reads supporting them in your dataset. This command will apply several filters and write a new vcf with `.recode` added at the end of the file name 

In [ ]:
vcftools --vcf subset_snps_no_repeats.vcf \
--min-meanDP 15 \
--max-meanDP 200 \
--min-alleles 2 \
--max-alleles 2 \
--max-missing 0.5 \
--recode \
--out subset_snps_no_repeats_filtered

**Question: At the end of this command it will tell you how many SNPs were retained. How many were removed due to this filtering?**

24) Now that we have a cleaned up file, let's calculate how many reads we have for each SNP in each sample on average using the `--depth` option of the program `vcftools`. Run the code below.

In [ ]:
vcftools --vcf subset_snps_no_repeats_filtered.recode.vcf --depth --out subset_snps_no_repeats_filtered

25) The above command created a new file called `subset_snps_no_repeats_filtered.idepth` that contains the average number of reads at each SNP in the file (column is named `MEAN_DEPTH`)

**Question: Look at this file and report the mean sequencing depth of your two samples, then explain in your own words what this number means.**

**Question: how does this number compare to the sequencing coverage that you calculated in your original bam file near the beginning of class?**

26) We are going to stop here today with our filtered vcf file, but before we go, do a mini writing assignment:  

**Question: Write a one-paragraph methods summary that you might include in a paper if you were writing this work from today up for a paper. Your paragraph should describe the important steps that were done and the software that was used.  You do not need to describe the steps that were just checking on things. But you should describe the SNP calling and filterning steps. Your lab score today will primarily be determined by whether this paragraph is compelte and accurate**


### Check back over your work and make sure you have answered every question and have a complete methods section before submitting! 

When you are done, you can print this as a PDF then upload to the Blackboard assignment for this week's lab. 